In [1]:
import torch
import timm
import backbone
import anchor
import importlib
import dataset
import cv2
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch import ToTensorV2




c:\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Python\Python313\Lib\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [12]:
importlib.reload(dataset)

image_dir = "dataset\\image\\"
label_dir = "dataset\\label\\"
classes = {0:'background',1:'NG',2:'OK'}

image_test =dataset.ListDataset(image_dir,label_dir,classes,transform=True)

train_transform = A.Compose([
    A.RandomSizedBBoxSafeCrop(width=450,height=450,erosion_rate=0.8),
    A.HorizontalFlip(p=0.3),
    A.VerticalFlip(p=0.2),
], bbox_params=A.BboxParams(format='yolo', # Specify input format
                           label_fields=['class_labels'] # Specify label argument name(s)
                            ))

final_transform = A.Compose([
    A.AdditiveNoise(noise_type="gaussian",
                    spatial_mode="constant",
                    noise_params={"mean_range": (0.0, 0.0), "std_range": (0.05, 0.15)}),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=1.0),
    A.Affine(translate_percent={'x': (-0.2, 0.2), 'y': (-0.2, 0.2)},  # x축 10~20%, y축 -20~20% 랜덤 이동
             p=0.3),
    A.Affine(rotate = (-20,20), p = 0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
], bbox_params=A.BboxParams(format='yolo', # Specify input format
                           label_fields=['class_labels'], # Specify label argument name(s)
                           ))

aug_para ={"alpha":0.7,"lambda":0.7,"prob":0.5}


image_test.add_agumentation(train_transform,aug_para,final_transform)

set_size = len(image_test)
train_dataset_size = int(0.8*set_size)
test_dataset_size = set_size-train_dataset_size

train_indces, valid_indces = torch.utils.data.random_split(range(set_size), [train_dataset_size, test_dataset_size])
print("set size: {0}, test_set size : {1}".format(train_dataset_size,test_dataset_size))


train_dataset = torch.utils.data.Subset(image_test, train_indces)
valid_dataset = torch.utils.data.Subset(image_test, train_indces)

#지금 문제는 torch.float32의 이미지 자료형이 
#엔코더에 입력변수로 이미지로 넣었을때 np.array하고 torch.tensor 
# 간에 충돌이 일어나면서 발생하고 있음

for i in range(len(train_dataset)):
    print("current index of dataset : {0}".format(train_indces[i]))
    try:
        data = train_dataset[i]
        print(f"[{i}] success: {data['image'].shape}, {len(data['bboxes'])} boxes")
    except Exception as e:
        print(f"[{i}] failed with error: {e}")
        print("labels:", train_dataset.labels[i])
        print("boxes :", train_dataset.boxes[i])
        break  # or continue if you want to scan all

batch_size = 200
num_workers = 1

train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               num_workers=num_workers,
                                               collate_fn = image_test.collate_fn)

loader_iter = iter(train_dataloader)
batch = next(loader_iter)

image, loc, class_target = batch
print(image.shape, loc.shape, class_target.shape)

'''
print(result["image"].shape)
print(result["image"].dtype)
print(result["bboxes"])
print(result["class_label"])

img = result["image"]
result_label = result["class_label"]

for i, box in enumerate(result["bboxes"]):
    x_center, y_center, w, h = box
    x1 = int((x_center - w/2) * 450)
    y1 = int((y_center - h/2) * 450)
    x2 = int((x_center + w/2) * 450)
    y2 = int((y_center + h/2) * 450)
    if result_label[i] == 1:
        cv2.rectangle(img, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2)
    else:
        cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)
plt.imshow(img)
plt.xticks([]); plt.yticks([])
plt.show()
'''

set size: 1084, test_set size : 272
current index of dataset : 797
selected augmentation: mix up
MIXUP BBOX CHECK
before cut: [0.09333252161741257, 0.3660964071750641, 0.0533333346247673, 0.029914790764451027]
before cut: [0.17333415150642395, 0.34188035130500793, 0.04999837279319763, 0.028490375727415085]
before cut: [0.23999999463558197, 0.35185232758522034, 0.04999837279319763, 0.029914790764451027]
before cut: [0.31333211064338684, 0.34330475330352783, 0.04333211109042168, 0.02421608939766884]
before cut: [0.3933337330818176, 0.33903151750564575, 0.04666707292199135, 0.029914790764451027]
before cut: [0.4333345592021942, 0.34188035130500793, 0.04333211109042168, 0.029914790764451027]
before cut: [0.4766668677330017, 0.3447291851043701, 0.03666585311293602, 0.02421608939766884]
before cut: [0.5300002694129944, 0.35042789578437805, 0.04999847337603569, 0.03276361897587776]
before cut: [0.753334641456604, 0.3589743673801422, 0.06333435326814651, 0.035612449049949646]
before cut: [0.72

AttributeError: 'Subset' object has no attribute 'labels'

In [ ]:
importlib.reload(anchor)

anchoring = anchor.DataEncoder((450,450),classes=[0,1])


test_out,test_cls = anchoring.encoder(gt_boxes,gt_classes)

print(test_cls)

unique, counts = test_cls.unique(return_counts=True)
for u, c in zip(unique, counts):  
    print(f"클래스 {int(u)}: {int(c)}개")


print(13*13*3,26*26*3,52*52*3)

In [ ]:
importlib.reload(dataset)

image_dir = "dataset\\image\\"
label_dir = "dataset\\label\\"
classes = {0:'background',1:'NG',2:'OK'}

image_test =dataset.ListDataset(image_dir,label_dir,classes)

In [ ]:
import albumentations as A

image,boxes, label=image_test[0]
print(len(image_test))
print(label)
print(image.shape)
print(boxes)
print(label)

In [ ]:


label_lst = ['NG','OK']

train_transform = A.Compose([
    A.RandomSizedBBoxSafeCrop(width=450,height=450,erosion_rate=0.8),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
], bbox_params=A.BboxParams(format='yolo', # Specify input format
                           label_fields=['class_labels'] # Specify label argument name(s)
                           ))

augmented = train_transform(image=image,bboxes=boxes,class_labels=label)

img = augmented['image']
result_label = augmented['class_labels']

print(img.dtype)
for i, box in enumerate(augmented['bboxes']):
    x_center, y_center, w, h = box
    x1 = int((x_center - w/2) * 450)
    y1 = int((y_center - h/2) * 450)
    x2 = int((x_center + w/2) * 450)
    y2 = int((y_center + h/2) * 450)
    if result_label[i] == 1:
        cv2.rectangle(img, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2)
    else:
        cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)
plt.imshow(img)
plt.xticks([]); plt.yticks([])
plt.show()

In [ ]:
import numpy as np
class MixUp:
    def __init__(self, alpha=0.7, prob=0.5):
        self.alpha = alpha
        self.prob = prob

    def __call__(self, datsset1, dataset2):
        img1 = np.array(datsset1['image']) / 255
        img2 = np.array(dataset2['image']) / 255
        mixed_img = (self.alpha * img1 + (1-self.alpha) * img2)
        mixed_boxes = datsset1['bboxes'] + dataset2['bboxes'] 
        mixed_labels = datsset1['class_labels'] + dataset2['class_labels']

        return mixed_img, mixed_boxes, mixed_labels

image_A,boxes_A,label_A=image_test[0]
image_B,boxes_B,label_B=image_test[100]
augmentedA = train_transform(image=image_A,bboxes=boxes_A,class_labels=label_A)
augmentedB = train_transform(image=image_B,bboxes=boxes_B,class_labels=label_B)

aug2 = MixUp(alpha=0.5, prob=1)
m_1,m_2,m_3 = aug2(augmentedA,augmentedB)

img = m_1
result_label = m_3

for i, box in enumerate(m_2):
    x_center, y_center, w, h = box
    x1 = int((x_center - w/2) * 450)
    y1 = int((y_center - h/2) * 450)
    x2 = int((x_center + w/2) * 450)
    y2 = int((y_center + h/2) * 450)
    if result_label[i] == 1:
        cv2.rectangle(img, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2)
    else:
        cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)
plt.imshow(img)
plt.xticks([]); plt.yticks([])
plt.show()

In [ ]:
import numpy as np

def apply_transform(transform, dataset, indices):
    """
    transform: Albumentations transform 객체 (예: train_transform)
    dataset: [(image, boxes, labels), ...] 형태의 리스트 or 커스텀 Dataset
    indices: 4개의 인덱스를 가진 리스트 (예: [0, 100, 10, 75])

    return: list of transformed outputs [(image, boxes, labels), ...]
    """
    results = []
    for idx in indices:
        image, boxes, labels = dataset[idx]
        augmented = transform(image=image, bboxes=boxes, class_labels=labels)
        results.append((augmented['image'], augmented['bboxes'], augmented['class_labels']))
    return results

augmented_outputs = apply_transform(train_transform,image_test,[10,20,30,40])
images = [out[0] for out in augmented_outputs]
bboxes = [out[1] for out in augmented_outputs]
labels = [out[2] for out in augmented_outputs]

Mosic_test = dataset.SimpleMosaic(225,True)
m_1,m_2,m_3 = Mosic_test(image=images, bboxes=bboxes, class_labels=labels)

img = m_1
result_label = m_3

print(img.shape)

for i, box in enumerate(m_2):
    x_center, y_center, w, h = box
    x1 = int((x_center - w/2) * 450)
    y1 = int((y_center - h/2) * 450)
    x2 = int((x_center + w/2) * 450)
    y2 = int((y_center + h/2) * 450)
    if result_label[i] == 1:
        cv2.rectangle(img, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2)
    else:
        cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)
plt.imshow(img)
plt.xticks([]); plt.yticks([])
plt.show()

In [ ]:
import numpy as np
importlib.reload(dataset)

image_A,boxes_A,label_A=image_test[0]
image_B,boxes_B,label_B=image_test[99]
augmentedA = train_transform(image=image_A,bboxes=boxes_A,class_labels=label_A)
augmentedB = train_transform(image=image_B,bboxes=boxes_B,class_labels=label_B)

aug2 = dataset.Cutmix(lamda=0.5, prob=1)
m_1,m_2,m_3 = aug2(augmentedA,augmentedB)

img = m_1
result_label = m_3

for i, box in enumerate(m_2):
    x_center, y_center, w, h = box
    x1 = int((x_center - w/2) * 450)
    y1 = int((y_center - h/2) * 450)
    x2 = int((x_center + w/2) * 450)
    y2 = int((y_center + h/2) * 450)
    if result_label[i] == 1:
        cv2.rectangle(img, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2)
    else:
        cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)
plt.imshow(img)
plt.xticks([]); plt.yticks([])
plt.show()